In [11]:
import random
from collections import deque
from board import Board, Tile


In [7]:
# Define Player class
class Player:
    def __init__(self, id, start_position, cards):
        self.id = id
        self.position = start_position
        self.cards = deque(cards)  # Use deque for easy card management
        self.current_card = self.cards[0] if self.cards else None

    def rotate_excess_tile(self, tile, rotation):
        tile.orientation = (tile.orientation + rotation) % 360
        return tile

    def select_push_location(self, direction, push_position):
        return push_position

    def move_piece(self, new_position):
        self.position = new_position

    def __repr__(self):
        return f"Player({self.id}, Position: {self.position}, Current Card: {self.current_card}, Remaining Cards: {list(self.cards)})"


In [8]:
class LabyrinthEnv:
    def __init__(self, board):
        self.board = board
        self.player = None
        self.last_exit_position = None  # Track the last exit position
        self.reset()


    def reset(self):
        # Deal cards to the player
        cards = ['A', 'B', 'C', 'D', 'E']  # Example card deck
        random.shuffle(cards)
        self.player = Player(id=1, start_position=(0, 0), cards=cards)
        self.board.add_player(self.player)
        self.last_exit_position = None


    def add_player(self, player):
        self.players.append(player)


    def step(self, player_id, action):
        # Handle the action (push tile, move player)
        if action['type'] == 'push':
            self.handle_push(action)
        elif action['type'] == 'move':
            self.handle_move(action)


    def handle_push(self, action):
        direction = action['direction']
        push_position = action['position']

        # Check if the push is valid
        if self.is_invalid_push(direction, push_position):
            raise ValueError("Invalid push location: No pushback allowed.")

        # Perform the push and update the last exit position
        exit_position = self.board.push_tile(direction, push_position)
        self.last_exit_position = (direction, exit_position)


    def is_invalid_push(self, push_position):
        # Check if the push is at the last exit position
        return self.last_exit_position == push_position


    def handle_move(self, action):
        new_position = action['new_position']
        if self.is_valid_move(self.player.position, new_position):
            self.player.move(new_position)
            tile = self.board.tiles[new_position[0]][new_position[1]]
            if tile.token:
                self.player.collect_token(tile.token)


    def is_valid_move(self, current_position, new_position):
        # Implement logic to check if the move is valid (contiguous path)
        return True  # Placeholder for actual logic


    def get_player_by_id(self, player_id):
        for player in self.players:
            if player.id == player_id:
                return player
        return None
    

    def render(self):
        self.board.visualize_board()
        for player in self.players:
            print(f"Player {player.id} Position: {player.position}")
            print(f"Player {player.id}'s current card: {player.cards[0] if player.cards else 'No more cards'}")

    def is_done(self):
        return all(len(player.cards) == 0 for player in self.players)


In [13]:
# Define main function for testing
def main():
    # Create a board
    board = Board(size=7)
    
    # Initialize the game environment
    env = LabyrinthEnv(board)
    
    # Reset the environment
    env.reset()
    
    # Add the player to the environment
    cards = ['A', 'B', 'C', 'D', 'E']
    player = Player(id=1, start_position=(0, 0), cards=cards)
    env.add_player(player)
    
    # Display the initial state of the player
    print(env.players[0])
    
    tile = Tile(orientation=0)
    rotated_tile = player.rotate_excess_tile(tile, 90)
    print(f"Rotated Tile: {rotated_tile}")
    
    # Select a push location
    try:
        push_location = player.select_push_location('right', (1, 3))
        print(f"Push Location: {push_location}")
    except ValueError as e:
        print(e)
    
    # Push the tile onto the board
    try:
        env.step(player.id, {'type': 'push', 'direction': 'right', 'position': (1, 3)})
        print("Push successful")
    except ValueError as e:
        print(e)
    
    try:
        env.step(player.id, {'type': 'push', 'direction': 'left', 'position': (1, 3)})  # This should raise an error
    except ValueError as e:
        print(e)
    
    # Move the player piece
    env.step(player.id, {'type': 'move', 'new_position': (1, 1)})
    print(env.players[0])

if __name__ == "___MAIN__":
    main()